In [1]:
import cv2
import os
import time

# Define dataset path
dataset_path = "family_dataset"
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

# Ask for the family member's name
member_name = input("Enter the family member's name (max 10 members allowed): ").strip()
member_folder = os.path.join(dataset_path, member_name)

if not os.path.exists(member_folder):
    os.makedirs(member_folder)

# Initialize webcam
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

count = 0  # Track the number of saved images

while count < 10:  # Capture 10 images per family member
    ret, frame = cap.read()
    if not ret:
        print("Error: Webcam access failed.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        face_image = frame[y:y + h, x:x + w]

        # Save image with a timestamp
        image_name = f"{member_name}_{int(time.time())}.jpg"
        cv2.imwrite(os.path.join(member_folder, image_name), face_image)
        print(f"✅ Saved image {count + 1} for {member_name}")

        count += 1  # Increment count

    cv2.imshow("Capturing Faces", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print(f"✅ All 10 images saved for {member_name} in {member_folder}")


Enter the family member's name (max 10 members allowed):  shristi


✅ Saved image 1 for shristi
✅ Saved image 2 for shristi
✅ Saved image 3 for shristi
✅ Saved image 4 for shristi
✅ Saved image 5 for shristi
✅ Saved image 6 for shristi
✅ Saved image 7 for shristi
✅ Saved image 8 for shristi
✅ Saved image 9 for shristi
✅ Saved image 10 for shristi
✅ All 10 images saved for shristi in family_dataset\shristi


In [7]:
import cv2
import os
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Define dataset path
dataset_path = "family_dataset"

# Prepare data
X = []  # Images
y = []  # Labels (Family Member Names)

label_map = {}  # Maps names to numeric labels
current_label = 0

# Read images for each family member
for person in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person)
    
    if os.path.isdir(person_path):
        label_map[current_label] = person  # Assign label to name
        
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (100, 100))  # Resize to standard size
            X.append(img.flatten())  # Convert image to 1D array
            y.append(current_label)
        
        current_label += 1  # Next label

X = np.array(X)
y = np.array(y)

# Train KNN model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Save trained model & label map
with open("face_recognition_model.pkl", "wb") as f:
    pickle.dump(knn, f)
with open("label_map.pkl", "wb") as f:
    pickle.dump(label_map, f)

print("✅ Training complete! Model saved as face_recognition_model.pkl")


✅ Training complete! Model saved as face_recognition_model.pkl


In [9]:
import cv2
import os
import pickle
import numpy as np
from tensorflow.keras.models import load_model
import time

# Load trained face recognition model
with open("face_recognition_model.pkl", "rb") as f:
    knn = pickle.load(f)
with open("label_map.pkl", "rb") as f:
    label_map = pickle.load(f)

# Load UTKFace-trained age prediction model
age_model = load_model("age_model.h5")

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Cross-check age with stabilized value
stable_age = 0
previous_age = 0
age_predictions = []  # List to store age predictions for stabilization
content_restricted = False  # Variable to track if content is restricted

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_image = gray[y:y + h, x:x + w]
        face_resized = cv2.resize(face_image, (100, 100)).flatten().reshape(1, -1)
        
        # Recognize face
        label_pred = knn.predict(face_resized)[0]
        name = label_map.get(label_pred, "Unknown")
        
        # Predict age using UTKFace model
        face_rgb = cv2.resize(frame[y:y + h, x:x + w], (64, 64)) / 255.0
        face_rgb = np.expand_dims(face_rgb, axis=0)
        predicted_age = int(age_model.predict(face_rgb)[0][0])

        # Stabilize age prediction (using moving average)
        age_predictions.append(predicted_age)
        if len(age_predictions) > 5:  # Only consider last 5 predictions for stability
            age_predictions.pop(0)
        
        stable_age = int(np.mean(age_predictions))  # Calculate the stabilized age

        # Display results
        text = f"{name} - Age: {stable_age}"
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        # Check for content restriction (if age < 18)
        if name != "Unknown" and stable_age < 18:
            content_restricted = True
        else:
            content_restricted = False

    # Show webcam feed with the results
    cv2.imshow("Face Recognition & Age Detection", frame)

    # Exit condition
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # Print the final result when 'q' is pressed
        print(f"\n🔎 Final Predicted Age: {stable_age}")
        
        if content_restricted:
            print("🔴 Content is Restricted for this user.")
        else:
            print("✅ No Content Restriction for this user.")

        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━